In [1]:
# pip install langchain-experimental pandas tabulate

In [1]:
import pandas as pd
from langchain_community.llms import Ollama   # # from langchain_ollama import OllamaLLM  # # TODO 차후 최신 OllamaLLM 로 변경 필요
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# from langchain.agents import AgentType  # # from langchain.agents.agent_types import AgentType

# 1. 로컬 LLM 설정 (Mistral Small)
# [중요] Agent가 정확한 코드 생성위해 temperature=0 설정
llm = Ollama(model="mistral-small", temperature=0)
# llm = OllamaLLM(model="mistral-small", temperature=0)

# 2. 분석위한 dummy data TODO csv나 DB에서 실데이터 불러오는 것으로 수정
data = {
    'date': pd.date_range(start='2025-10-01', periods=14),
    'visitor_count': [1200, 1300, 1100, 900, 850, 1500, 1600, 1150, 1050, 950, 800, 1450, 1550, 1100],
    'order_count': [120, 135, 115, 80, 75, 180, 195, 110, 100, 85, 70, 175, 190, 105],
    'ad_spend': [500, 500, 500, 300, 300, 700, 700, 500, 500, 500, 300, 700, 700, 500],
    'platform': ['PC', 'Mobile', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC']
}
df = pd.DataFrame(data)

# 3. Pandas DataFrame Agent 생성
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True, # Agent가 생각 과정(Thought/Action) 화면 출력
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # # 로컬 환경에서 LLM이 파이썬 코드 실행 허용 설정
)

# 4. 분석 요청 프롬프트 (Few-shot 구조 결합)
query = """
제공된 데이터프레임을 기반으로 다음 구조에 맞춰 한국어 보고서를 작성하세요:

1. 데이터 요약: 전체적인 수치(방문자, 주문수 등) 요약
2. 가설 설정: 플랫폼(PC/Mobile)별 효율 차이나 요일별 특징에 대한 가설
3. 분석 결과: 가설을 검증하기 위해 실제 코드를 실행한 통계적 근거 (예: 플랫폼별 전환율 비교)
4. 비즈니스 제언: 분석 결과를 바탕으로 한 마케팅 전략

반드시 코드를 실행하여 정확한 평균값과 상관관계를 계산한 뒤 답변하세요.
"""

print("Agent GO!GO!GO!...")
print("-" * 50)

try:
    response = agent.run(query)
    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")

/tmp/ipykernel_2985119/884807818.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral-small", temperature=0)
/tmp/ipykernel_2985119/884807818.py:46: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = agent.run(query)


🤖 에이전트가 데이터를 탐색하고 코드를 실행 중입니다...
--------------------------------------------------


> Entering new AgentExecutor chain...
Thought:
I need to perform several steps to generate the report:

1. Summarize the overall metrics (visitor count, order count, etc.).
2. Set hypotheses about platform efficiency and day-of-week characteristics.
3. Analyze the data to verify these hypotheses using statistical measures like conversion rates and correlations.
4. Provide business recommendations based on the analysis.

Action: python_repl_ast
Action Input:
```python
import pandas as pd

# Summarize overall metrics
total_visitors = df['visitor_count'].sum()
total_orders = df['order_count'].sum()
total_ad_spend = df['ad_spend'].sum()

print(f"Total Visitors: {total_visitors}")
print(f"Total Orders: {total_orders}")
print(f"Total Ad Spend: {total_ad_spend}")

# Calculate conversion rates
df['conversion_rate'] = df['order_count'] / df['visitor_count']

# Calculate platform-wise metrics
platform_metrics =